In [1]:
# Checking the version of PYTHON
import sys
if sys.version_info < (3,0):
    raise Exception('Please use Python version 3 or greater.')
    
# useful additional packages 
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np


import sys
sys.path.append('/Users/sabreitweiser/qiskit-sdk-py/') #Change to point to your qiskit root

# importing the QISKit
from qiskit import QuantumCircuit, QuantumProgram
import Qconfig

# import basic plot tools
from qiskit.tools.visualization import plot_histogram